# Install pytorch

In [14]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision

# Import library

In [0]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

# Use GPU and CUDA
use_cuda = True
if use_cuda and torch.cuda.is_available():
    net.cuda()

# Parameters

In [0]:
input_size    = 784   # The image size = 28 x 28 = 784
hidden_size   = 500   # The number of nodes at the hidden layer
num_classes   = 10    # The number of output classes. In this case, from 0 to 9
num_epochs    = 5     # The number of times entire dataset is trained
batch_size    = 100   # The size of input data took for one iteration
learning_rate = 1e-3  # The speed of convergence

# Dataset

In [7]:
train_dataset = dsets.MNIST(root='./data/MNIST/',
                           train=True,
                           transform=transforms.ToTensor(),
                           download=True)

test_dataset = dsets.MNIST(root='./data/MNIST/',
                           train=False,
                           transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

Processing...
Done!


# Define network

In [0]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    # Inherited from the parent class nn.Module
        self.fc1 = nn.Linear(input_size, hidden_size)  # 1st Full-Connected Layer: 784 (input data) -> 500 (hidden node)
        self.relu = nn.ReLU()                          # Non-Linear ReLU Layer: max(0,x)
        self.fc2 = nn.Linear(hidden_size, num_classes) # 2nd Full-Connected Layer: 500 (hidden node) -> 10 (output class)
    
    def forward(self, x):                              # Forward pass: stacking each layer together
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Neural network
net = Net(input_size, hidden_size, num_classes)

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

# Run

In [11]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):   # Load a batch of images with its (index, data, class)
        images = Variable(images.view(-1, 28*28))         # Convert torch tensor to Variable: change image from a vector of size 784 to a matrix of 28 x 28
        labels = Variable(labels)
        
        if use_cuda and torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
        
        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        outputs = net(images)                             # Forward pass: compute the output class given a image
        loss = criterion(outputs, labels)                 # Compute the loss: difference between the output class and the pre-given label
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes
        
        if (i+1) % 100 == 0:                              # Logging
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))


Epoch [1/5], Step [100/600], Loss: 0.2960
Epoch [1/5], Step [200/600], Loss: 0.2419
Epoch [1/5], Step [300/600], Loss: 0.2308
Epoch [1/5], Step [400/600], Loss: 0.1639
Epoch [1/5], Step [500/600], Loss: 0.1725
Epoch [1/5], Step [600/600], Loss: 0.2014
Epoch [2/5], Step [100/600], Loss: 0.1205
Epoch [2/5], Step [200/600], Loss: 0.1004
Epoch [2/5], Step [300/600], Loss: 0.0954
Epoch [2/5], Step [400/600], Loss: 0.0361
Epoch [2/5], Step [500/600], Loss: 0.0570
Epoch [2/5], Step [600/600], Loss: 0.0512
Epoch [3/5], Step [100/600], Loss: 0.1283
Epoch [3/5], Step [200/600], Loss: 0.1491
Epoch [3/5], Step [300/600], Loss: 0.1276
Epoch [3/5], Step [400/600], Loss: 0.0560
Epoch [3/5], Step [500/600], Loss: 0.0332
Epoch [3/5], Step [600/600], Loss: 0.0583
Epoch [4/5], Step [100/600], Loss: 0.0307
Epoch [4/5], Step [200/600], Loss: 0.1919
Epoch [4/5], Step [300/600], Loss: 0.0128
Epoch [4/5], Step [400/600], Loss: 0.0388
Epoch [4/5], Step [500/600], Loss: 0.0545
Epoch [4/5], Step [600/600], Loss:

# Test

In [12]:
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    
    if use_cuda and torch.cuda.is_available():
        images = images.cuda()
        labels = labels.cuda()
    
    
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)  # Choose the best class from the output: The class with the best score
    total += labels.size(0)                    # Increment the total count
    correct += (predicted == labels).sum()     # Increment the correct count
    
print('Accuracy of the network on the 10K test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10K test images: 97 %


# Save

In [0]:
# torch.save(net.state_dict(), 'fnn_model.pkl')